In [17]:
import csv
import pandas as pd

In [18]:
user_file = '/home/dipp/Github/Master-Thesis-dipp/USER_LOCATION_REPLACEMENT/userAll_sample_500 copy.csv'
location_file = '/home/dipp/Github/Master-Thesis-dipp/USER_LOCATION_REPLACEMENT/frequentplaces1814031_sample.txt'
location_resolved_file = '/home/dipp/Github/Master-Thesis-dipp/USER_LOCATION_REPLACEMENT/frequentplaces1814031_sample.txt.resolved'
result_file = '/home/dipp/Github/Master-Thesis-dipp/USER_LOCATION_REPLACEMENT/new_res_sample_.csv'

In [19]:
def is_equal_hashing(a, b, hashing=hash):
    return len(a) == len(b) and hashing(a) == hashing(b) and a == b

In [20]:

with open(location_file) as lf:
    loc_data = lf.read()
    loc_data_dict = {}
    i = 0
    for l in loc_data.split("\n"):
        loc_data_dict[i] = l.lower()
        i+=1


with open(location_resolved_file) as lrf:
    loc_res_data = lrf.read()
    loc_res_data_dict = {}
    i = 0
    for l in loc_res_data.split("\n"):
        loc_res_data_dict[i] = l
        i+=1

In [21]:
from itertools import islice

print(dict(islice(loc_data_dict.items(), 5)))
print(dict(islice(loc_res_data_dict.items(), 5)))

{0: 'los angeles, ca', 1: 'chicago, il', 2: 'venezuela', 3: 'new york', 4: 'berlin'}
{0: "['US', 'United States', 'CA', 'California', 'Los Angeles']", 1: "['US', 'United States', 'IL', 'Illinois', 'Chicago']", 2: "['VE', 'Venezuela', None, None, None]", 3: "['US', 'United States', 'NY', 'New York', 'New York']", 4: "['DE', 'Germany', 'Berlin', 'Berlin', 'Berlin']"}


In [22]:
cols = ['createdAt', 'followers', 'favourites', 'location', 'userId']
user_data_dic = dict.fromkeys(cols)
no_match = "['none', 'none', 'none', 'none', 'none']"

In [23]:
with open(user_file, newline='', encoding='utf-8') as uf:
    user_data = csv.DictReader((x.replace('\0', '') for x in uf), fieldnames=cols)
    No_of_Data = 0
    No_of_Match = 0
    No_of_No_Match = 0
    user_data_dict_list = []
    for line in user_data:
        No_of_Data += 1
        counter = 0
        for key, value in loc_data_dict.items():
            
            try:
                if(is_equal_hashing(line['location'].lower(), value) == True):
                    counter = 1
                    line['location'] = loc_res_data_dict[key]
                    user_data_dict_list.append(line)
                    No_of_Match += 1
                    

            except AttributeError:
                line['location'] = no_match
                user_data_dict_list.append(line)
                counter = 1
                No_of_No_Match +=1
                

            
        if (counter == 0):
            line['location'] = no_match
            user_data_dict_list.append(line)
            No_of_No_Match += 1
        
        counter = 1


print('No. Of Data: ', No_of_Data)
print('No. of Match: ', No_of_Match)
print('No. of No Match', No_of_No_Match)

        
            

No. Of Data:  503
No. of Match:  323
No. of No Match 180


In [24]:
df = pd.DataFrame(user_data_dict_list)
df=df[df.columns.dropna()]
#df.drop('INVALID', axis=1, inplace=True) 
df.to_csv(result_file, index=False)